# <center> Gestion des processus et des ressources par un système d'exploitation </center>

<img src='intro.png' style='float:right;' width=500>

* Considérons une utilisation banale d'un ordinateur:
Un utilisateur rédige un rapport dans un traitement de texte, plusieurs onglets d'un navigateur sont ouverts pour effectuer des recherches, un logiciel d'édition d'image permet de modifier des illustrations pour le rapport, tout cela en écoutant de la musique via une application de l'ordinateur.

* Tous ces programmes donnent l'impression de s'éxécuter en même temps.
* Pourtant, un programme est une suite d'instructions en langage machine qui sont exécutées une à une par le processeur. Comment éxécuter en même temps les instructions du traitement de texte et du lecteur de musique ?
* Les systèmes d'exploitations modernes permettent les exécutions concurrentes de plusieurs programme grâce à l'ordonnanceur, ce que nous allons découvrir dans ce document après avoir rappelé les principes d'éxécution d'un programme. 
* Nous découvrirons également quelques commandes permettant d'afficher et arrêter des processus.



## 1. Exécution des programmes (Rappels)
* Un éxécutable est un fichier qui contient une suite d'instructions en langage machine(appéles souvent des fichiers binaires de l'anglais *binary*). C'est donc une suite d'octets que le processeur est capble de décoderet éxécuter.
* Ainsi lorsque l'on éxécute un programme ( en double cliquant sur son icône ou à l'aide d'une console), le système d'exploitation effectue les actions suivantes :
   * Le fichier éxécutable est copié dans la mémoire vive (RAM) à une certaine adresse.
   * Le système d'exploitation écrit la valeur de cette adresse dans un registre.
   
* Afin de synchroniser toutes les éxécutions, le processeur dispose d'une horloge dont la fréquence tourne actuellement autour de 3 Ghz.Cette fréquence indique le nombre de cycles que le processeur effectue par seconde, c'est à dire 3 milliards d'éxécution par seconde !

* Il est important de comprendre qu'a chaque cycle, une seule instruction est exécutée. Ainsi, pour simplifier, un programme est éxécuté instruction par instruction à chaque cycle.

## 2. L'ordonnanceur
* Cette description est néanmoins incomplète, car il faudrait donc attendre qu'un programme ait fini d'éxécuter toutes ses instructions avant de pouvoir passer à un autre programme.

##### Exemple :
Considérons ce petit programme python :

```
a=input('Entrez une valeur`)
print('Bravo')
```

Avec la description précédente, il serait impossible de faire autre chose pour l'utilisateur puisque le progamme ne fait rien d'autre tant qu'il n'a pas saisi une valeur.


* Pour pallier à ce problème pouvoir éxécuter plusieurs programmes "en même temps", les systèmes d'exploitationmodernes utilisent des *interruptions* .

### Interruptions
* Une interruption est un signal envoyé au processeur lorsqu'un évènement se produit. Les interruptions peuvent être générées par un matériel (le disque dur signale qu'il a fini d'écrire des octets, une carte réseau signale l'arrivée de données, etc...).
* Le processeur qui reçoit ce signal interrompt l'éxécution en cours, mémorise l'état de cette éxécution, puis choisit un autre programme à éxécuter parmi ceux qui sont en attente.
* Ce sont ces interruptions qui permettent aux programmes de s'éxécuter de façon concurrente ("en même temps").

### Vocabulaire
Avant d'aller plus loin, quelques termes qui seront utilisés par la suite :

* __Exécutable :__ 
  * Un fichier binaire contenant des instructions machines directement exécutables par un processeur.

* __Processus :__
  * Un programme en cours d'éxécution. Chaque processus est identifié par un numéro de manière unique par le processeur.

* __Thread ou tâche :__
  * Exécution d'une suite d'instructions d'un processus :
     * Deux processus = Deux programmes (par exemple un navigateur et un traitement de texte)
     * Deux threads peuvent être issus d'un même processus et s'éxécutent donc de manière concurrente (par exemple dans un navigateur : télécharger un fichier et afficher un texte dans un onglet)

* __Exécution concurrente :__
  * Deux processus ou deux threads s'éxécutent de manière concurrentes lorsqu'ils essaient de "s'éxécuter en même temps". 

* __Exécution parallèle :__
  * Deux processus ou deux threads s'éxécutent de manière parallèles lorsqu'ils s'éxécutent réellement en même temps. Ceci n'est possible qu'avec la présence de plusieurs processeurs sur la machine (architecture multicoeurs).

* __Ressource:__
  * Tout objet informatique matériel ou logiciel nécessaire à l’exécution d’un programme. Exemples de ressources :
     * fichiers, mémoire, processus, imprimante, carte son ,...


### Ordonnancement
* Le système d'exploitation va donc gérer l'ordre d'éxécution des processus, à l'aide d'un programme appelé *gestionnaire d'interruption*.


* Voici un exemple de déroulement :
  1. Le programme traitement de texte est en cours d'éxécution( saisie de texte, mise en page,...)
  2. Une interruption d'horloge se déclenche.
  3. Le gestionnaire d'interruption est appelé, prend la main,  et sauvegarde l'état du traitement de texte à un endroit particulier de la mémoire.
  4. Il choisit dans la liste des processus un autre processus, par exemple le navigateur web.
  5. Il restaure l'état de ce processus dans lequel il était au moment de sa dernière interruption.
  6. Le gestionnaire d'interruption rend la main au navigateur web, jusqu'à la prochaine interruption.


* Afin de pouvoir choisir parmi tous les processus à lequel exécuter lors de la prochaine interruption, le système d'exploitation conserve entre autre pour chaque processus :
  * Son numero unique , le PID (Process ID).
  * L'état dans lequel se trouve le processus (en attente/en cours d'éxécution)
  * La valeur des registres lors de sa dernière interruption (contenu des variables,...)
  * ...


* Les processus peuvent ainsi être placés dans une file(au sens de celui déjà abordé en programmation). Le premier processus dans la file sera le prochain à reprendre son éxécution, puis sera placé à la fin de la file à la prochaine interruption.Ceci évite qu'un processus monopolise les ressources de calcul du processeur.


* On peut visualiser les processus présents dans un système à l'aide de son gestionnaire de tâches.
Ci-dessous, les gestionnaires de tâches de Windows 10(à gauche) et d'une distribution Linux Debian(à droite) en mode graphique ou console :
<img src='processgui.jpg' style='float:center;' width=800>
<img src='process.png' style='float:center;' width=800>


### Etats des processus
Les processus peuvent donc être dans différents états :

* __Nouveau :__
Le processus est en cours de création, le système d'exploitation vient de copier l'éxécutable en mémoire et initialise ses informations.


* __Prêt :__
Le processus peut être le prochain à s'éxécuter, il est dans la file des processus qui "attendent" leur tour.


* __En éxécution (ou élu) :__
Le processus est en train de s'éxécuter.


* __En attente :__
Le processus a été interrompu et est en attente d'une prochaine interruption.


* __Terminé:__
Le processus est terminé et le système d'exploitation l'enlève des processus identifiés.


### Cycle de vie d'un processsus
On peut résumer ces différents états dans le schéma ci-dessous :

<img src='cycle.png' style='float:center;' width=600>


##### Remarque :
 * Les états "nouveau" et "terminé" sont éphémères. Le plus souvent, l'état d'un processus varie entre "prêt", "en attente" et "en cours d'éxécution".
 * Quelque soit son état, un processus peut se terminer de façon anormale (erreur matérielle, intervention de l'utilisateur...)
 * Les interruptions peuvent être déclenchées par le processeur à intervalles réguliers, mais aussi par un matériel (une carte son qui demande un accès au micro,...)


###### Exercice 1 :  Il se trouve dans le dossier <a href='Exercices/Processus_Exercices.pdf' target='_blank'>Exercices</a>

## 3. Quelques commandes

<img src='command.png' style='float:center;' width=600>

Il est possible à l'utilisateur d'identifier les différents processus à l'oeuvre dans le système voire d'agir dessus, que ce soit à l'aide d'une interface graphique où à l'aide de commandes.

### Afficher les processus 
<img src='ps.png' style='float:right;' width=600>

##### Exemple 1 :

* Voici un résultat de la commande `ps -a -u -x` dans un environnement Linux.
  * L'argument `-a` permet d'afficher les processus de tous les utilisateurs.
  * `-u` d'afficher le nom de l'utilisateur plutôt qu'un identifiant numérique.
  * `-x` d'afficher les processus principaux.
  
  
  
* Pour en savoir plus sur cette commande et ses résultats, on pourra consulter l'aide à l'aide de la commande `man ps`



* Quelques observations :
  * La colonne `USER` indique le nom de l'utilisateur pour lequel a été lancé le processus.
  * La colonne `PID` donne l'identifiant numérique  du processus.
  * Les colonnes `%CPU` et `%MEM` indiquent respectivement le taux d'occupation du processeur et de la mémoire par le processus/
  * La colonne `STAT` indique l'état du processus (la première lettre en majuscule):
     * `R` : *running* ou *runnable*, respectivement en éxécution ou prêt (la commande `ps` ne fait pas la différence).
     * `S` : *sleeping* , en attente.
  * Les colonnes `START` et `TIME` indiquent respectivement la date/heure à laquelle le programme a été lancé et le temps total pendant lequel le processus a été dans l'état "en éxécution".
  * La colonne `COMMAND` indique le nom du programme, plus précisément la commande qui a lancé le programme.
  
### Terminer des processus

L'utilisateur peut demander l'interruption d'un processus à l'aide de la commande `kill`, en spécifiant le ou les numéros de processus (c'est à dire le(s) PID).

##### Exemple 2 :
* La commande `kill 921 938` envoie un signal de terminaison aux deux processus listés par les numéros données :
   * Cette demande peut être interceptée et réalisée par l'application concernée(ici le traitement de texte libre office), cela équivaut à fermer la fenêtre de celle-ci. Dans ce cas , il sera demandé à l'utilisateur s'il veut sauvegarder son travail.
   * Il est également possible de terminer le processus sans passer  par l'application concernée (ici avec l'option `-9` en saisissant `kill -9 921 938`). Dans ce cas, c'est le système qui termine le processus sans que l'utiilsateur puisse sauvegarder l'état. C'est ce qui se passe par exemple lorsque l'application "ne répond plus".
   
### Remarques :
* Sous Linux, on peut aussi utiliser la commande `top` pour afficher les processus.
* Dans un environnement Windows, on peut utiliser la commande `tasklist` pour afficher les processus et `taskkill` pour terminer un processus.
<img src='tasklist.png' style='float:center;' width=600>


###### Exercice 2 :  Il se trouve dans le dossier <a href='Exercices/Processus_Exercices.pdf' target='_blank'>Exercices</a>

## 4. Interblocage
* Ainsi , les processus d'un système s'éxécutent de manière concurrente, dans un ordre hors de leur contrôle, défini par l'ordonnanceur du système d'exploitation. Ce fonctionnement possède de nombreux avantages :
  * Cela permet à un grand nombre de programmes de fonctionner "en même temps" à l'aide d'un seul processeur.
  * Les ressources sont optimisées(économie de calcul, d'énergie...)



* Cependant, lorsqu'un processus est interrompu, ce n'est pas de son fait, ce n'est pas lui qui décide.Il reprend plus tard son éxécution , dans l'état ou il était avant l'interruption :
   * Cela ne pose pas de problème tant que le processus n'a pas besoin de ressources externes à lui même.
   * Mais lorsque'un processus doit accéder à des ressources partagée (écrire dans un fichier comme dans l'exercice 1, ou accéder à un matériel comme un carte son), cela peut poser problème :
      * En particulier, lorsqu'une même ressource est attendue par plusieurs processus et qu'elle ne peut être utilisée par un seul, un blocage est alors possible: On appelle cela le phénomène d'interblocagge, ou verrou mortel (*deadlock*)

##### Exemple 3 :
Certaines ressources matérielles sont en accès exclusif: Cela signifie qu'elle ne peuvent être utilisée que par au plus un seul processus. Dans un ordinateur, c'est le cas de la carte son.

Imaginons deux processus  :
* `record` : C'est un processus qui accède au micro de la carte son, et qui écrit le son enregistré dans une mémoire tampon de 10 secondes.
* `play` : C'est un processus qui accède au haut parleur de la carte son, et qui lit le contenu qu'il reçoit en entrée.

Classiquement, on utilise le processus `record` pour enregistrer du son,  mémorisé ensuite dans un fichier, que nous appellerons `sound.mp3`. Puis on utilise le processus `play` pour lire le fichier et diffuser le son.

Supposons maintenant que l'on essaie d'exécuter le processus `record` en le dirigeant directement vers le processus `play`, sans passer par l'écriture de `sound.mp3` :
 * Le processus `record` accède à la carte son et enregistre du son. 
 * Le processus `play` essaie alors d'accéder à la carte son mais se retrouve bloqué en attente que le processus`record` libère la carte son et ne peut jouer aucun son.
 * Le processus `record` se retrouve également bloqué car il ne peut pas enregistrer plus que le contenu de la mémoire tampon qui se retrouve pleine.
 
Les deux processus sont alors en interblocage:
 * `play` attend que la carte son soit libre pour pouvoir lire ce qu'il reçoit en entrée.
 * `record` attend que sa mémoire tampon se vide pour libérer la carte son.

### Conditions de Coffman

<img src='coffman.jpg' style='float:right;' width=150>

* L'interblocage est le danger de la concurrence des processus dans un système monoprocesseur.
* L'informaticien Edward Grady Coffman Jr (1934-...) a le premier, en 1971, décrit quatre conditions nécéssaires à la présence d'un interblocage, appelée conditions de coffman:
  * Exclusion mutuelle : Au moins une ressource est en accès exclusif
  * Rétention et attente : Un processus détient une ressource et demande une autre ressource détenue par un autre processus.
  * Non préemption : une ressource ne peut être rendue que par un processus qui la détient (sinon le système prendrait lui même la décision de libérer cette ressource)
  * Attente circulaire : les processus s'attendent mutuellement.
  
  
* Plusieurs stratégies existent pour éviter, détecter ou résoudre l'interblocage. Dans la plupart des utilisations des systèmes d'exploitation, la solution la plus souvent utilisée est de terminer les processus concernés lorsqu'ils sont en interblocage (par exemple à l'aide de `kill`)

###### Exercice 3 :  Il se trouve dans le dossier <a href='Exercices/Processus_Exercices.pdf' target='_blank'>Exercices</a>